In [1]:
# init
%matplotlib notebook

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib as mpl
from IPython.display import display, HTML
import time


def nop(*args, **kwargs):
    pass
rng = np.random.default_rng()

zipfdic = {}
def zipf_pmf(x, n):
    """
    Calculate the value of the Zipf PMF at a given x for n categories.
    
    Parameters:
    x (int): The rank of the category.
    n (int): The total number of categories.
    
    Returns:
    float: The value of the Zipf PMF at the given rank x.
    """
    if(n in zipfdic):
        if(x in zipfdic[n]):
            return zipfdic[n][x]
    else:
        zipfdic[n] = {}
    
    # Calculate the normalization constant
    harmonic_sum = sum([1 / i for i in range(1, n+1)])
    c = 1 / harmonic_sum
    
    # Calculate the value of the Zipf PMF at rank x
    zipfdic[n][x] = c * (1 / x)
    
    return zipfdic[n][x]

# direction vectors for each ID: 0:right, 1:down, 2:left, 3:down 
# adding 1 = rotate right; subtract 1 = rotate left
directions = [ (1, 0), (0, 1), (-1, 0), (0, -1) ]

def perp(d):
    return d[1], d[0]

# v1 + v2
def addvec(x1, y1, x2, y2):
    return x1+x2, y1+y2

def addvec(v1, v2):
    return v1[0]+v2[0], v1[1]+v2[1]

# scal * v
def scalevec(scal, x, y):
    return scal*x, scal*y

def scalevec(scal, v):
    return scal*v[0], scal*v[1]

default_rnd_genes = 24
sight_range = 12

In [31]:
# note: commented indexes  and order are wrong; change later

# 0: dormant sense
def dormant(obj, env):
    return False

# sense food ahead radial
def snfdfw_r(obj, env):
    d = directions[obj.dir]
    p = perp(d)
    lo, hi = 0, 0
    for i in range(1, sight_range+1):
        for j in range(lo, hi+1):
            coords = addvec(scalevec(i, d), scalevec(j, p))
            if env.isFood(*addvec((obj.x, obj.y), coords)):
                return True
        lo-=1
        hi+=1
    return False

# sense food right radial
def snfdri_r(obj, env):
    d = directions[(obj.dir+1)%4]
    p = perp(d)
    lo, hi = 0, 0
    for i in range(1, sight_range+1):
        for j in range(lo, hi+1):
            coords = addvec(scalevec(i, d), scalevec(j, p))
            if env.isFood(*addvec((obj.x, obj.y), coords)):
                return True
        lo-=1
        hi+=1
    return False

# sense food left radial
def snfdlf_r(obj, env):
    d = directions[(obj.dir-1)%4]
    p = perp(d)
    lo, hi = 0, 0
    for i in range(1, sight_range+1):
        for j in range(lo, hi+1):
            coords = addvec(scalevec(i, d), scalevec(j, p))
            if env.isFood(*addvec((obj.x, obj.y), coords)):
                return True
        lo-=1
        hi+=1
    return False

# 10: stack == 0
def sneqz(obj, env):
    return obj.stack == 0

# 11: stack > 0
def sngtz(obj, env):
    return obj.stack > 0

# 12: stack < 0
def snltz(obj, env):
    return obj.stack < 0

# 17: stack <= neighbor ahead
def sncmpn(obj, env):
    tile = addvec((obj.x, obj.y), directions[obj.dir])
    if(not env.emptyTile(*tile) and not env.outofbounds(*tile)):
        return obj.stack <= env.tiles[tile].stack
    return False

# 13: wall ahead
def snwlfw(obj, env):
    return env.outofbounds(*addvec((obj.x, obj.y), directions[obj.dir]))

# 14: wall behind
def snwlbh(obj, env):
    return env.outofbounds(*addvec((obj.x, obj.y), directions[(obj.dir+2)%4]))

# 15: wall to left
def snwllf(obj, env):
    return env.outofbounds(*addvec((obj.x, obj.y), directions[(obj.dir-1)%4]))

# 16: wall to right
def snwlri(obj, env):
    return env.outofbounds(*addvec((obj.x, obj.y), directions[(obj.dir+1)%4]))

# 18: sense of individual ahead
def sninfw(obj, env):
    tile = addvec((obj.x, obj.y), directions[obj.dir])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Individual)

# 19: sense of individual behind
def sninbh(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir+2)%4])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Individual)

# 20: sense of individual to the left
def sninlf(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir-1)%4])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Individual)

# 21: sense of individual to the right
def sninri(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir+1)%4])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Individual)
    
# 18: sense of food ahead
def snfdfw(obj, env):
    tile = addvec((obj.x, obj.y), directions[obj.dir])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Environment.FoodObject)

# 19: sense of food behind
def snfdbh(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir+2)%4])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Environment.FoodObject)

# 20: sense of food to the left
def snfdlf(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir-1)%4])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Environment.FoodObject)

# 21: sense of food to the right
def snfdri(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir+1)%4])
    if not (env.emptyTile(*tile)):
        return isinstance(env.tiles[tile], Environment.FoodObject)

def notsn(sn):
    return lambda obj, env: not sn(obj, env)

# list containing boolean sensory functions for each sense ID
senses = [sneqz, sngtz, snltz, sncmpn,
          snwlfw, snwlbh, snwllf, snwlri, 
          snfdfw, snfdbh, snfdlf, snfdri,
          snfdfw_r, snfdlf_r, snfdri_r,
          sninfw, sninbh, sninlf, sninri]
senses = senses+list(map(notsn, senses))
senses.append(dormant)

# 0: try move ahead
def mvfw(obj, env):
    tile = addvec((obj.x, obj.y), directions[obj.dir])
    if(env.emptyTile(*tile) and not env.outofbounds(*tile)):
        env.tiles[tile] = obj
        env.tiles[obj.x, obj.y] = None
        obj.x = tile[0]
        obj.y = tile[1]

# 1: try move backward
def mvbk(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir+2)%4])
    if(env.emptyTile(*tile) and not env.outofbounds(*tile)):
        env.tiles[tile] = obj
        env.tiles[obj.x, obj.y] = None
        obj.x = tile[0]
        obj.y = tile[1]

# 2: try move left
def mvlf(obj, env):    
    tile = addvec((obj.x, obj.y), directions[(obj.dir-1)%4])
    if(env.emptyTile(*tile) and not env.outofbounds(*tile)):
        env.tiles[tile] = obj
        env.tiles[obj.x, obj.y] = None
        obj.x = tile[0]
        obj.y = tile[1]

# 3: try move right
def mvri(obj, env):
    tile = addvec((obj.x, obj.y), directions[(obj.dir+1)%4])
    if(env.emptyTile(*tile) and not env.outofbounds(*tile)):
        env.tiles[tile] = obj
        env.tiles[obj.x, obj.y] = None
        obj.x = tile[0]
        obj.y = tile[1]


# 4: attack/eat ahead
def attack(obj, env):
    tile = addvec((obj.x, obj.y), directions[obj.dir])
    if(not env.emptyTile(*tile) and not env.outofbounds(*tile)):
        env.tiles[tile].on_attacked(obj, env)
        
# 5: increment stack
def incstack(obj, env):
    obj.stack+=1
    
# 6: decrement stack
def decstack(obj, env):
    obj.stack-=1
    
# 7: negate stack
def negstack(obj, env):
    obj.stack = -obj.stack
    
# 8: turn right
def tnri(obj, env):
    obj.dir = (obj.dir+1)%4

# 9: turn left
def tnlf(obj, env):
    obj.dir = (obj.dir-1)%4
    
# 10: turn around backwards
def tnbk(obj, env):
    obj.dir = (obj.dir+2)%4

# list containing behavior functions for each behavior ID
behaviors = [mvfw, mvbk, mvlf, mvri, attack, incstack, decstack, negstack, tnri, tnlf, tnbk]

def generate_rnd_genome(num_genes):
    genes = []
    for i in range(num_genes):
        sense = rng.integers(len(senses))
        behavior = rng.integers(len(behaviors))
        genes.append((sense, behavior))
    return genes

In [32]:
# class defining an evolution environment
class Environment:
    class EnvObject:
        def __init__(self, x, y, canwin):
            self.x = x
            self.y = y
            self.canwin = canwin
            self.rank = -1
            self.ranker = None
            self.score = 0
            self.stack = 0
            self.dir = rng.integers(4)
        def behavior(self, env):
            pass
        def on_attacked(self, attacker, env):
            pass
        def mutate(self, env):
            pass
        
    class FoodObject(EnvObject):
        def __init__(self, x, y):
            super().__init__(x, y, False)
        def on_attacked(self, attacker, env):
            attacker.score+=1
            env.remove_obj(self)
    
    def __init__(self, size, winfunc, ranker, reverse_ranking=False, objects=[]):
        self.tiles = np.empty(shape=size, dtype=Environment.EnvObject) # tile grid
        self.size = size # size of environment
        self.winfunc = winfunc # Individual, Environment -> bool | chooses whether Individual can reproduce
        self.objects = [] # objects in environment
        self.ranker = ranker
        self.reverse_ranking = reverse_ranking
        self.successful = []
        self.time = 0
        self.generations = 0
        self.redundant_gens = 0
        if(objects != []):
            for obj in objects:
                self.insert_obj(obj)
            
    def reset(self):
        self.tiles = np.empty(shape=self.size, dtype=Environment.EnvObject)
        self.objects = []
    
    def insert_obj(self, obj):
        if(obj.canwin): 
            obj.ranker = self.ranker
            self.objects.append(obj)
        self.tiles[obj.x, obj.y] = obj
        
    def remove_obj(self, obj):
        self.tiles[obj.x, obj.y] = None
        if(obj.canwin):
            self.objects.remove(obj)
        del obj
        
    def populate_area_random(self, n, w, h, num_genes=default_rnd_genes, lf=0, tp=0):
        i=0
        while i < n:
            x = np.random.randint(lf, lf+w)
            y = np.random.randint(tp, tp+h)
            if(self.tiles[x,y] != None):
                continue
            
            indi = Individual(generate_rnd_genome(num_genes), x, y)
            self.insert_obj(indi)
            i+=1
            
    def place_food_random(self, n, w, h, lf=0, tp=0):
        i = 0
        while i < n:
            x = np.random.randint(lf, lf+w)
            y = np.random.randint(tp, tp+h)
            if(self.tiles[x,y] != None):
                continue
            
            indi = self.FoodObject(x, y)
            self.insert_obj(indi)
            i+=1
    
    def run(self):
        for o in self.objects:
            o.behavior(self)
        self.time+=1
    
    def try_mutations(self, p=.05):
        for o in self.objects:
            o.mutate(p)
        
    def get_successful(self):
        return list(filter(lambda o: self.winfunc(o, self), 
                           filter(lambda o: o.canwin, self.objects)))
    
    def ranksort(self):
        return sorted(filter(lambda p: p.rank != -1, self.get_successful()), 
                      key=lambda p: p.rank, reverse=self.reverse_ranking)
    
    def repopulate(self, n, w, h, lf=0, tp=0, num_genes=default_rnd_genes):
        successful = self.ranksort()
        self.reset()
        i=0
        count = len(successful)
        if(count == 0):
            self.redundant_gens+=1
            successful = self.successful
            count = len(successful)
        dist = None
        if(count > 1):
            self.successful = successful
            dist = [zipf_pmf(count, i) for i in range(1, count+1)]
            dist = [p + (1-sum(dist))/count for p in dist]
        while i < n:
            x = np.random.randint(lf, lf+w)
            y = np.random.randint(tp, tp+h)
            if(self.tiles[x,y] != None):
                continue
            
            genes = []
            if count == 0:
                genes = generate_rnd_genome(num_genes)
            elif count == 1:
                genes = successful[0].genes
            else:
                genes = rng.choice(successful[:count], p=dist).genes
            ind = Individual(genes, x, y)
            self.insert_obj(ind)
            i+=1
            
    def repopulate_from_genes(self, genes, n, w, h, dist=None, lf=0, tp=0):
        self.reset()
        i=0
        count = len(genes)
        if(count > 1 and dist == None):
            dist = [zipf_pmf(count, i) for i in range(1, count+1)]
            dist = [p + (1-sum(dist))/count for p in dist]
        while i < n:
            x = np.random.randint(lf, lf+w)
            y = np.random.randint(tp, tp+h)
            if(self.tiles[x,y] != None):
                continue
            
            g = []
            if count == 0:
                raise Exception("Cannot repopulate from empty population gene set")
            elif count == 1:
                g = successful[0].genes
            else:
                g = rng.choice(genes, p=dist).genes
            ind = Individual(g, x, y)
            self.insert_obj(ind)
            i+=1
    
    def generation(self, frames, sidechain=nop, post=nop, p=.05, 
                   repop=0, repopw=0, repoph=0, rplf=0, rptp=0):
        '''
        Run the environment then allow winning bacteria to reproduce with mutation.
        
        int frames: number of frames to run for before reproduction
        function sidechain: function that runs after every frame
        function post: function that runs after generation
        float p: probability of mutation
        int repop: number of children to repopulate
        int repopw, repoph, rplf, rptp: width, height, left, top of simulation             
        '''
        for i in range(frames):
            sidechain()
            self.run()
        sidechain()
        
        self.repopulate(repop, repopw, repoph, rplf, rptp)
        self.try_mutations(p)
        self.time = 0
        post()
        self.generations+=1
        
    def outofbounds(self, x, y):
        return not (x >= 0 and x < self.size[0] and y >= 0 and y < self.size[1])
    def emptyTile(self, x, y):
        if not self.outofbounds(x, y):
            return self.tiles[x,y] == None
        return True
    def isFood(self, x, y):
        if not self.emptyTile(x, y):
            return isinstance(self.tiles[x, y], Environment.FoodObject)
        return False

In [33]:
# class defining an individual in the simulation with a certain genome
class Individual(Environment.EnvObject):
    def __init__(self, genes, x, y):
        super().__init__(x, y, True)
        self.genes = genes # list of sense id : behavior id tuples

    def behavior(self, env):
        for (s, b) in self.genes:
            # for each activated sense, perform behavior connected to it
            if(senses[s](self, env)):
                behaviors[b](self, env)
        if(env.winfunc(self, env)):
            self.rank = self.ranker(self, env)
    
    def on_attacked(self, attacker, env):
        pass
    
    # mutates genes with mutation probability p
    def mutate(self, p):
        if(rng.random() < p):
            i = rng.choice(range(len(self.genes)))
            choice = rng.choice((0,1))
            if(choice == 0):
                self.genes[i] = (rng.integers(len(senses)), self.genes[i][1])
            else:
                self.genes[i] = (self.genes[i][0], rng.integers(len(behaviors)))
            
        

In [34]:
plt.ioff()

def tilesToImg(env):
    def classify(o):
        if o == None:
            return 0
        elif o.canwin:
            if(env.winfunc(o, env)):
                return 3
            else:
                return 2
        else:
            return 1
    return np.vectorize(classify)(env.tiles)

def createColormap(colormap_list):
    cmap = mpl.colors.ListedColormap(colormap_list)
    cmap.set_bad(color='w', alpha=0)
    return cmap

# First set up the figure, the axis, and the plot element we want to animate
def animate_environment(env: Environment, frames: int, cmap=mpl.colormaps["binary"]):
    fig, ax = plt.subplots()
    ax.set_xlim(0, env.size[0])
    ax.set_ylim(1, env.size[1])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    # initialization function: plot first state
    def init():
        ax.imshow(tilesToImg(env), cmap=cmap, vmin=0, vmax=3)
        return ax

    # animation function
    def animate(i=0):
        env.run()
        ax.clear()
        ax.imshow(tilesToImg(env), cmap=cmap, vmin=0, vmax=3)
        return ax

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, interval=100, blit=True)
    
    env.time = 0
    video = anim.to_html5_video()
    html = HTML(video)
    display(html)
    plt.close()

In [35]:
# run env for n generations
def run_n_generations(env, n, frames=100, repop=80, repopw=80, repoph=20, sidechain=nop, post=nop, p=.05, tp=0, lf=0):
    total = time.time()
    for i in range(n):
        env.generation(frames, sidechain=sidechain, repop=repop, repopw=repopw, repoph=repoph, rplf=lf, rptp=tp, post=post)
    total = time.time()-total
    return total, total/n

In [36]:
# environment with food condition
def foodWinfunc(food):
    return lambda ind, env: ind.score >= food

num_patches = 16
food_per_patch = 20
patches = {}
def foodplacepatches():
    patches = set()
    i = 0
    while i < num_patches:
        pair = tuple(rng.choice(( (rng.choice((0, 7)), rng.integers(0, 7)), (rng.integers(0, 7), rng.choice((0, 7))) )))
        
        if(pair not in patches):
            patches.add(pair)
            food_env.place_food_random(food_per_patch, 10, 10, lf=pair[0]*10, tp=pair[1]*10)
            i+=1 

total_food = 320
def foodplace():
    food_env.place_food_random(total_food, 80, 20, tp=60)

food_env = Environment((80,80), foodWinfunc(4), lambda ind,env: ind.score, True)
foodplace()
food_env.populate_area_random(100, 80, 20, lf=0, tp=0)

In [37]:
checkpoint_genes = dict()

In [38]:
generations = 100
frames = 200

total, avg = run_n_generations(food_env, generations, frames=frames, p=.15, post=foodplace, repop=100, repopw=80, repoph=20)
print("total time for " + str(generations) + " generations: " + str(total) + " | average generation time: " + str(avg))

checkpoint_genes[food_env.generations] = food_env.successful

total time for 100 generations: 2330.9337866306305 | average generation time: 23.309337866306304


In [41]:
# animate food env
animate_environment(food_env, 200, cmap=createColormap(['w', 'k', 'r', 'g']))
print(len(food_env.get_successful()))
food_env.repopulate(100, 80, 20, lf=0, tp=0)
food_env.try_mutations(.15)
food_env.generations+=1
foodplace()
print(food_env.generations)

0
102


In [ ]:
checkpoint_genes[food_env.generations] = list(map(lambda s: s.genes, food_env.successful))

In [27]:
food_env.repopulate_from_genes(checkpoint_genes[50], 100, 80, 20, lf=0, tp=0)
foodplace()

In [29]:
print(checkpoint_genes.keys())

dict_keys([50, 151, 255])


In [40]:
print(food_env.redundant_gens)

78
